In [387]:
import pandas as pd
import numpy as np
from datetime import datetime
import os
from pathlib import Path 

In [388]:
'''
Pipeline para consegur las progresiones de un mes Puntial, teniendo en cuenta la SC. Las progresiones se mostraran por Formato, por Tienda, Sector, Seccion y grupo de Familia.
'''

'\nPipeline para consegur las progresiones de un mes Puntial, teniendo en cuenta la SC. Las progresiones se mostraran por Formato, por Tienda, Sector, Seccion y grupo de Familia.\n'

# Carga de Archivos y transformaciones generales

In [389]:
df_ventas_y_volumen = pd.read_csv('data/ventas - agosto.csv', encoding='utf-16', header=1)
df_debitos = pd.read_csv('data/debitos - agosto.csv', encoding='utf-16', header=1, sep=',', decimal=',')
padron = pd.read_excel('data/padron - agosto.xlsx', header=17)

## Trabajo sobre Ventas y Volumen

In [390]:
#Me quedo unicamente con las columnas importantes
df_ventas_y_volumen = df_ventas_y_volumen[['Año', 'Mes', 'Direccion', 'Punto Operacional', 'Sector', 'Seccion', 'Grupo de Familia', 'Ventas c/impuesto', 'Venta en Unidades']]

#Renombro las columnas
df_ventas_y_volumen.columns = (df_ventas_y_volumen.columns.str.strip().str.lower().str.replace(" ", "_"))
df_ventas_y_volumen.rename(columns={
    'ventas_c/impuesto':'venta',
    'venta_en_unidades':'volumen'
}, inplace=True)

#Genero una columna para Obtener el ID tienda
df_ventas_y_volumen['numero_operacional'] = df_ventas_y_volumen['punto_operacional'].str.split('-').str[0]

#Me quedo con las columnas necesarias
ventas = df_ventas_y_volumen[['año', 'mes', 'direccion', 'numero_operacional', 'punto_operacional', 'sector', 'seccion', 'grupo_de_familia', 'venta']]
volumen = df_ventas_y_volumen[['año', 'mes', 'direccion', 'numero_operacional', 'punto_operacional', 'sector', 'seccion', 'grupo_de_familia', 'volumen']]

#Quito los NA de las columans de valores
ventas.dropna(subset=['venta'], how='any', inplace=True)
volumen.dropna(subset=['volumen'], how='any', inplace=True)

#Realizo transformaciones para quitar carateres y convertir las columnas a valores numericos
ventas['venta'] = ventas['venta'].str.replace('.','').str.replace(',','.').astype('float')
volumen['volumen'] = volumen['volumen'].str.split(',').str[0].str.replace('.','').astype('int')

#Renombro las columnas con valores de ambos DF
ventas.rename(columns={
    'venta':'valores'
}, inplace=True)

volumen.rename(columns={
    'volumen':'valores'
}, inplace=True)

#Categorizo los valores tanto de volumne como de Ventas
ventas['categoria'] = 'VCT'
volumen['categoria'] = 'VOL'

#Agrupo las ventas
ventas_agrupado = ventas.groupby(['año', 'mes', 'direccion', 'numero_operacional', 'punto_operacional', 'categoria'])['valores'].sum().reset_index()

#Quito Envases del Volumen y Agrupo
volumen_sin_vol = volumen[~volumen['grupo_de_familia'].str.contains('ENVASES')]
volumen_agrupado = volumen_sin_vol.groupby(['año', 'mes', 'direccion', 'numero_operacional', 'punto_operacional', 'categoria'])['valores'].sum().reset_index()

C:\Users\juan_mera\AppData\Local\Temp\ipykernel_20268\394968719.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ventas.dropna(subset=['venta'], how='any', inplace=True)
C:\Users\juan_mera\AppData\Local\Temp\ipykernel_20268\394968719.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  volumen.dropna(subset=['volumen'], how='any', inplace=True)
C:\Users\juan_mera\AppData\Local\Temp\ipykernel_20268\394968719.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.py

## Trabajo sobre Debitos

In [391]:
#Renombro el DF
debitos_agrupados = df_debitos
#Renombro las columnas como corresponden
debitos_agrupados.columns = debitos_agrupados.columns.str.lower().str.replace(' ','_')
debitos_agrupados.rename(columns={
    'cant._tickets_por_local':'valores'
}, inplace=True)
#Renombro la columna de Debitos a valores
debitos_agrupados['categoria'] = 'DEB'
#Genero una columna Categorica
debitos_agrupados['numero_operacional'] = debitos_agrupados['punto_operacional'].str.split('-').str[0]
#Genero columna para el ID tienda
debitos_agrupados = debitos_agrupados[['año', 'mes', 'direccion', 'numero_operacional', 'punto_operacional', 'categoria', 'valores']]
#Quito nulos numericos de la columna valores
debitos_agrupados.dropna(subset=['valores'], how='any', inplace=True)
#Convierto la columna de valores a su tipo de datos correspondiente
debitos_agrupados['valores'] = debitos_agrupados['valores'].str.replace('.','').astype(int)

C:\Users\juan_mera\AppData\Local\Temp\ipykernel_20268\3003591098.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  debitos_agrupados.dropna(subset=['valores'], how='any', inplace=True)
C:\Users\juan_mera\AppData\Local\Temp\ipykernel_20268\3003591098.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  debitos_agrupados['valores'] = debitos_agrupados['valores'].str.replace('.','').astype(int)


## Trabajo sobre el padron

In [392]:
#Cargo el padron
padron = pd.read_excel('data/padron - agosto.xlsx', header=17)
#Selecciono las columnas que me sirven del padron
padron = padron[['N°', 'NOMBRE', 'Fecha apertura', 'ORGANIZACIÓN ', 'M² SALÓN', 'M² PGC', 'M² PFT', 'M² BAZAR', 'M² Electro', 'M² Textil', 'M² Pls', 'M² GALERIAS', 'PROVINCIA', 'M² Parcking', 'FIN DE CIERRE', 'ENE.2', 'FEB.2', 'MAR.2', 'ABR.2', 'MAY.2', 'JUN.2', 'JUL.2', 'AGO.2', 'SEP.2', 'OCT.2', 'NOV.2', 'DIC.2']]
#Cambio de nombres en el padron
padron.columns = (
    padron.columns
    .str.lower()
    .str.strip()
    .str.replace(' ', '_', regex=False)
    .str.replace('m²', 'm', regex=False)
    .str.replace('.2','')
)
padron.columns
#Formateo la fecha para que tenga sentido
padron['fecha_apertura'] = padron['fecha_apertura'].dt.strftime('%d/%m/%Y')
#Cambio el nombre de la columna N por "Numero Operacional"
padron.rename(columns={'n°':'numero_operacional'}, inplace=True)
#Quito los valores nulos utilizando como referencia la columna Numero Operacional, nombre y fecha apertura
padron.dropna(subset=['numero_operacional', 'nombre', 'fecha_apertura', 'ago'], how='any', inplace=True)
#Genero una funcion para convertir los valores de una columna a mayuscula
def maysc(df: pd.DataFrame, columna: str):
    df[columna] = df[columna].str.upper()
    #Aplico la formula a la columna 'ago' para que todos los valores sean en mayuscula
maysc(padron, 'ago')
#Coloco el numero operacional como numero
padron['numero_operacional'] = padron['numero_operacional'].astype(int)

## Concateno todos los df (venta, debito y volumen) y lo joineo con el padron

In [393]:
df = pd.concat([ventas_agrupado, volumen_agrupado, debitos_agrupados])
#Convierto el ID a numero
df['numero_operacional'] = df['numero_operacional'].astype(int)

In [394]:
# Genero el Join del df Agupado con el Padron con el objetivo de quedarme unicamente con aquellas tiendas Comparables
df_join = df.merge(padron, how='left', on='numero_operacional')

## Trabajo sobre Progresiones Total Formato

In [395]:
#Me quedo unicamente con las columnas que me sirven del DF Joineado (ACA TENGO LA SC DEL MES)
df_join = df_join[['año', 'mes', 'direccion', 'numero_operacional', 'punto_operacional', 'fecha_apertura', 'fin_de_cierre', 'provincia','categoria', 'valores', 'ago']]
#Filtro unicamente las lineas que sean Superficie Comparable
df_join_sc = df_join[df_join['ago'] == 'SC']
df_progresiones_total_carrefour = df_join_sc.groupby(['año','categoria'])['valores'].sum().reset_index().pivot_table('valores', ['categoria'], 'año', 'sum').reset_index()
#Genero la Columna de Progresiones
df_progresiones_total_carrefour['progresion'] = round(((df_progresiones_total_carrefour[2025] / df_progresiones_total_carrefour[2024]) - 1) * 100, 1)


## Trabajo sobre Progresiones por Formato

In [396]:
#Agrupo el df por año, direccion y categoria para pivotrear y construir un df para realizar las progresiones
df_progresiones_formato = df_join_sc.groupby(['año', 'direccion','categoria'])['valores'].sum().reset_index().pivot_table('valores', ['direccion', 'categoria'], 'año', 'sum').reset_index()
#Genero la Columna de Progresiones
df_progresiones_formato['progresion'] = round(((df_progresiones_formato[2025] / df_progresiones_formato[2024]) - 1) * 100, 1)
#Pivoteo nuevamente la informacion para que este en un formato mas legible (Wide y no Long)
df_progresiones_formato = df_progresiones_formato.pivot_table([2024, 2025, 'progresion'], 'direccion', 'categoria').sort_values(by=('progresion', 'VOL'), ascending=False)

## Trabajo sobre Progresiones por Provincia

In [397]:
df_progresiones_provincia = df_join_sc.groupby(['año', 'provincia','categoria'])['valores'].sum().reset_index().pivot_table('valores', ['provincia', 'categoria'], 'año', 'sum').reset_index()
#Genero la Columna de Progresiones
df_progresiones_provincia['progresion'] = round(((df_progresiones_provincia[2025] / df_progresiones_provincia[2024]) - 1) * 100, 1)
#Pivoteo nuevamente la informacion para que este en un provincia mas legible (Wide y no Long)
df_progresiones_provincia = df_progresiones_provincia.pivot_table([2024, 2025, 'progresion'], 'provincia', 'categoria').sort_values(by=('progresion', 'VOL'), ascending=False)

## Trabajo sobre Progresiones por Tiendas / Formatos

In [398]:
#Agrupo el DF joineado con el padron y ya con la superficie comparable y agrego las tiendas
df_progresiones_tiendas = df_join_sc.groupby(['año', 'direccion', 'punto_operacional','categoria'])['valores'].sum().reset_index().pivot_table('valores', ['direccion', 'punto_operacional', 'categoria'], 'año', 'sum').reset_index()
#Genero las Progresiones
df_progresiones_tiendas['progresion'] = round(((df_progresiones_tiendas[2025] / df_progresiones_tiendas[2024]) - 1) * 100, 1)
#Pivoteo la Informacion para mostrar en unformato Wide (Mas legible) y no un un formato long (Mas estructura para trabajar)
df_progresiones_tiendas = df_progresiones_tiendas.pivot_table(values=[2024, 2025, 'progresion'], index=['direccion', 'punto_operacional'], columns='categoria').reset_index().sort_values(by=['direccion', ('progresion', 'VOL')], ascending=[False, False])

## Trabajo sobre Progresiones por Sector Total (Solo Vol y VCT porque Debitos llega hasta el detalle de Tiendas)

### Concateno las ventas y el volumen sin envases, lo cruzo con el padron, me quedo con los valores comparables segun el mes, genero tres df agrupados por sector, seccion y grupo de familia

In [399]:
#Concateno las ventas con el volumen sin Envases con el objetivo de agruparlo por sus distintas carecteristicas y  asi conseguir las progresiones totales por Sector, seccion y grupo de familia
df_venta_volumen = pd.concat([ventas, volumen_sin_vol])
#Convierto la columna Numero Operacional para realizar el merge con el padron
df_venta_volumen['numero_operacional'] = df_venta_volumen['numero_operacional'].astype(int) 
df_venta_volumen = df_venta_volumen.merge(padron, how='left', on='numero_operacional')
#Me quedo unicamente con las columnas que me sirven y los valores comparables
df_venta_volumen = df_venta_volumen[['año', 'mes', 'direccion', 'numero_operacional', 'punto_operacional', 'sector', 'seccion', 'grupo_de_familia','fecha_apertura', 'fin_de_cierre', 'provincia', 'categoria', 'valores', 'ago']]
df_venta_volumen = df_venta_volumen[df_venta_volumen['ago'] == 'SC'] 
#Agrupo por sector
df_venta_volumen_agrupado_sector = df_venta_volumen.groupby(['año', 'mes', 'direccion', 'numero_operacional', 'punto_operacional', 'sector', 'categoria'])['valores'].sum().reset_index()
#Agrupo por Seccion
df_venta_volumen_agrupado_seccion = df_venta_volumen.groupby(['año', 'mes', 'direccion', 'numero_operacional', 'punto_operacional', 'seccion', 'categoria'])['valores'].sum().reset_index()
#Agrupo por grupo de familia
df_venta_volumen_agrupado_grupo_familia = df_venta_volumen.groupby(['año', 'mes', 'direccion', 'numero_operacional', 'punto_operacional', 'grupo_de_familia', 'categoria'])['valores'].sum().reset_index()

## Trabajo sobre los Sectores

In [400]:
#Pivoteo la Info para generar las Progresiones
sectores_total = df_venta_volumen_agrupado_sector.groupby(['año', 'mes', 'direccion', 'sector', 'categoria'])['valores'].sum().reset_index().pivot_table(values='valores', index=['sector', 'categoria'], columns='año', aggfunc='sum').reset_index()
#Genero las progresiones por Sector
sectores_total['progresion'] = round(((sectores_total[2025] / sectores_total[2024])-1)*100,1)
#Pivoteo la Informacion para disponibilizar la informacion en formato wide y no long
progresion_sectores_total = sectores_total.pivot_table(values=[2024, 2025, 'progresion'], index='sector', columns='categoria', aggfunc='sum').sort_values(by=('progresion', 'VOL'), ascending=False)

## Trabajo sobre las secciones

In [401]:
#Pivoteo la Info para generar las Progresiones
seccion_total = df_venta_volumen_agrupado_seccion.groupby(['año', 'mes', 'direccion', 'seccion', 'categoria'])['valores'].sum().reset_index().pivot_table(values='valores', index=['seccion', 'categoria'], columns='año', aggfunc='sum').reset_index()
#Genero las progresiones por seccion
seccion_total['progresion'] = round(((seccion_total[2025] / seccion_total[2024])-1)*100,1)
#Pivoteo la Informacion para disponibilizar la informacion en formato wide y no long
progresion_seccion_total = seccion_total.pivot_table(values=[2024, 2025, 'progresion'], index='seccion', columns='categoria', aggfunc='sum').sort_values(by=('progresion', 'VOL'), ascending=False)

## Trabajo sobre los Grupos de Familia

In [402]:
#Pivoteo la Info para generar las Progrgrupo_de_familia
grupo_de_familia_total = df_venta_volumen_agrupado_grupo_familia.groupby(['año', 'mes', 'direccion', 'grupo_de_familia', 'categoria'])['valores'].sum().reset_index().pivot_table(values='valores', index=['grupo_de_familia', 'categoria'], columns='año', aggfunc='sum').reset_index()
#Genero las progresiones por grupo_de_familia
grupo_de_familia_total['progresion'] = round(((grupo_de_familia_total[2025] / grupo_de_familia_total[2024])-1)*100,1)
#Pivoteo la Informacion para disponibilizar la informacion en formato wide y no long
progresion_grupo_de_familia_total = grupo_de_familia_total.pivot_table(values=[2024, 2025, 'progresion'], index='grupo_de_familia', columns='categoria', aggfunc='sum').sort_values(by=('progresion', 'VOL'), ascending=False)

## Trabajo sobre Ventas y Vol Aperturado por Formato, Tienda, sector, seccion en una misma Tab

In [403]:
#Agrupo la informacion de las ventas y volumen por sector, seccion y GF. El problema aca es que en una misma tabla no puedo poner subtotales de sector seccion por tienda, por lo que tengo que generar tres tablas diferentes, cada una de estas aperturadas por Tienda y luego (Sector/seccion/GF)
df_aperturado = df_venta_volumen.groupby(['año', 'mes', 'direccion', 'numero_operacional', 'punto_operacional', 'sector', 'seccion', 'grupo_de_familia', 'categoria'])['valores'].sum().reset_index().pivot_table(values='valores', columns='año', index=['direccion', 'punto_operacional', 'sector', 'seccion', 'grupo_de_familia', 'categoria'], aggfunc='sum').reset_index()
#Obtengo la informacion correspondiente
df_tienda_sector = df_aperturado.groupby(['direccion', 'punto_operacional', 'sector', 'categoria'])[[2024, 2025]].sum()
df_tienda_seccion = df_aperturado.groupby(['direccion', 'punto_operacional', 'seccion', 'categoria'])[[2024, 2025]].sum()
df_tienda_grupo_de_familia = df_aperturado.groupby(['direccion', 'punto_operacional', 'grupo_de_familia', 'categoria'])[[2024, 2025]].sum()

In [404]:
#Calculo las progresiones
df_tienda_sector['progresion'] = round(((df_tienda_sector[2025] /df_tienda_sector[2024] -1) *100),1)
df_tienda_seccion['progresion'] = round(((df_tienda_seccion[2025] /df_tienda_seccion[2024] -1) *100),1)
df_tienda_grupo_de_familia['progresion'] = round(((df_tienda_grupo_de_familia[2025] /df_tienda_grupo_de_familia[2024] -1) *100),1)
#Reseteo los index
df_tienda_sector = df_tienda_sector.reset_index()
df_tienda_seccion = df_tienda_seccion.reset_index()
df_tienda_grupo_de_familia = df_tienda_grupo_de_familia.reset_index()
#Ordeno los valores por su volumen 
df_tienda_sector = df_tienda_sector.sort_values(by=['direccion', 'punto_operacional', 'sector', 'progresion'], ascending=[False, False, False, False])
df_tienda_seccion = df_tienda_seccion = df_tienda_seccion.sort_values(by=['direccion', 'punto_operacional', 'seccion', 'progresion'], ascending=[False, False, False, False])
df_tienda_grupo_de_familia = df_tienda_grupo_de_familia = df_tienda_grupo_de_familia.sort_values(by=['direccion', 'punto_operacional', 'grupo_de_familia', 'progresion'], ascending=[False, False, False, False])
#Pivoteo para presentar en un formato mas legible
df_tienda_sector = df_tienda_sector.pivot_table(values=[2024, 2025, 'progresion'], columns='categoria', index=['direccion', 'punto_operacional', 'sector'], aggfunc='sum').reset_index().sort_values(by=[('direccion',''), ('punto_operacional',    ''), ('sector',    ''), ('progresion', 'VOL')], ascending=[False, False, False, False])
df_tienda_seccion = df_tienda_seccion.pivot_table(values=[2024, 2025, 'progresion'], columns='categoria', index=['direccion', 'punto_operacional', 'seccion'], aggfunc='sum').reset_index().sort_values(by=[('direccion',''), ('punto_operacional',    ''), ('seccion',    ''), ('progresion', 'VOL')], ascending=[False, False, False, False])
df_tienda_grupo_de_familia = df_tienda_grupo_de_familia.pivot_table(values=[2024, 2025, 'progresion'], columns='categoria', index=['direccion', 'punto_operacional', 'grupo_de_familia'], aggfunc='sum').reset_index().sort_values(by=[('direccion',''), ('punto_operacional',    ''), ('grupo_de_familia',    ''), ('progresion', 'VOL')], ascending=[False, False, False, False])

## Exporto todas las tablas a Excel

In [405]:
try:
    out_dir = Path("results")
    out_dir.mkdir(parents=True, exist_ok=True)
    out_file = out_dir / f"Resultados.xlsx"

    with pd.ExcelWriter(out_file) as writer:
        df_progresiones_total_carrefour.to_excel(writer, sheet_name="Prog Carrefour - SC", index=False)
        df_progresiones_formato.to_excel(writer, sheet_name="Prog x Formatos - SC", index=True)
        df_progresiones_provincia.to_excel(writer, sheet_name="Prog x Provincia - SC", index=True)
        df_progresiones_tiendas.to_excel(writer, sheet_name="Prog x Tiendas - SC", index=True)
        progresion_sectores_total.to_excel(writer, sheet_name='Progresiones x Sector - SC', index=True)
        progresion_seccion_total.to_excel(writer, sheet_name='Progresiones x Seccion - SC', index=True)
        progresion_grupo_de_familia_total.to_excel(writer, sheet_name='Progresiones x GF - SC', index=True)
        df_tienda_sector.to_excel(writer, sheet_name='Prog Sector x Tienda - SC', index=True)
        df_tienda_seccion.to_excel(writer, sheet_name='Prog Seccion x Tienda - SC', index=True)
        df_tienda_grupo_de_familia.to_excel(writer, sheet_name='Prog GF x Tienda - SC', index=True)
        df_aperturado.to_excel(writer, sheet_name='Prog Aperturado x Tienda - SC', index=True)

    print(f"Archivo guardado en: {out_file}")

except Exception as e:
    print(f"No se logró guardar la información en un archivo Excel. ERROR: {e}")

Archivo guardado en: results\Resultados.xlsx
